# Random forest classification (multi-node, multi-GPU)

<table>
    <tr>
        <td>
            <img src="../_img/dask-horizontal.svg" width="300">
        </td>
        <td>
            <img src="../_img/rapids.svg" width="300">
        </td>
    </tr>
</table>

## Initialize A Dask Cluster

In [ ]:
from dask.distributed import Client, wait
from dask_saturn import SaturnCluster

n_workers = 3
cluster = SaturnCluster(
    n_workers=n_workers, scheduler_size="medium", worker_size="g4dnxlarge"
)
client = Client(cluster)
client

<div class="alert alert-success">
    Click that dashboard link to view some diagnostic information about the Dask cluster.
</div>

## Load data

In [ ]:
import dask_cudf

taxi = dask_cudf.read_csv(
    "s3://nyc-tlc/trip data/yellow_tripdata_2019-01.csv",
    parse_dates=["tpep_pickup_datetime", "tpep_dropoff_datetime"],
    storage_options={"anon": True},
    assume_missing=True,
)

print(f"Num rows: {len(taxi)}, Size: {taxi.memory_usage(deep=True).sum().compute() / 1e9} GB")

taxi.head()

## Prep for Training

Before training a model, we need to transform this dataset into a format that's better-suited to the research question. The function below does that with `cudf` operations. It also casts all of the input data to 32-bit types (`float32` for continuous variables, `int32` for integers).

<details><summary>(click here to learn why data scientists do this)</summary>

**Compute the Target**

The raw data don't contain a column that cleanly describes whether or not a particular ride could be considered a "high" tip. So we need to add one! In this exercise, a "high" tip is one that is greater than 20% of the fare amount.

**Add Features**

Giving a machine learning model a richer description of each training observation improves its ability to describe the relationship between those observations' characteristics and the target. These characteristics are called "features".

For example, instead of giving a model a raw timestamp, it can be valuable to provide multiple derived characteristics like hour of the day and day of the week. It's plausible, for example, that weekend rides might have a different distribution of tips because they tend to be for leisure, where weekday rides might be mostly people travelling for work.

**Remove Unused Features**

If the goal is to produce a model that could predict whether or not a tip will be high *at the beginning of the trip*, then characteristics that can only be known AFTER the tip have to be excluded. For example, you can't know the dropoff time or the type of payment until a ride has concluded.

Such features should be dropped before training.
    
</details>

In [ ]:
def prep_df(df: dask_cudf.DataFrame, target_col: str) -> dask_cudf.DataFrame:
    """
    Prepare a raw taxi dataframe for training.
        * computes the target ('high_tip')
        * adds features
        * removes unused features

    Casts all numeric values to 32-bit types, for efficiency and
    because some older versions of CUDA / ``cudf``, ``cuml``
    did not support 64-bit types in training data.
    """
    numeric_feat = [
        "pickup_weekday",
        "pickup_hour",
        "pickup_week_hour",
        "pickup_minute",
        "passenger_count",
    ]
    categorical_feat = [
        "PULocationID",
        "DOLocationID",
    ]
    features = numeric_feat + categorical_feat

    # add target
    df = df[df.fare_amount > 0]  # avoid divide-by-zero
    df["tip_fraction"] = df.tip_amount / df.fare_amount
    df[target_col] = df["tip_fraction"] > 0.2

    # add features
    df["pickup_weekday"] = df.tpep_pickup_datetime.dt.weekday
    df["pickup_hour"] = df.tpep_pickup_datetime.dt.hour
    df["pickup_week_hour"] = (df.pickup_weekday * 24) + df.pickup_hour
    df["pickup_minute"] = df.tpep_pickup_datetime.dt.minute

    # drop unused columns
    df = df[features + [target_col]].astype("float32").fillna(-1)

    # convert target to int32 for efficiency (it's just 0s and 1s)
    df[target_col] = df[target_col].astype("int32")

    return df

target_col = "high_tip"

taxi_train = prep_df(df=taxi, target_col=target_col)

Dask performs computations in a [lazy manner](https://tutorial.dask.org/01x_lazy.html), so we persist the dataframe to perform data loading and feature processing and load into GPU memory.

In [ ]:
%%time
taxi_train = taxi_train.persist()
_ = wait(taxi_train)

## Train a Model

Now that the data have been prepped, it's time to build a model!

For this task, we'll use the `RandomForestClassifier` from `cuml`. If you've never used a random forest or need a refresher, consult ["Forests of randomized trees"](https://scikit-learn.org/stable/modules/ensemble.html#forest) in the `sciki-learn` documentation.

The code below initializes a random forest classifier with the following parameter values.

* `n_estimators=100` = create a 100-tree forest
* `max_depth=10` = stop growing a tree once it contains a leaf node that is 10 levels below the root

All other parameters use the defaults from `RandomForestClassifier`.

<details><summary>(click here to learn why data scientists do this)</summary>

**Setting max_depth**

Tree-based models split the training data into smaller and smaller groups, to try to group together records with similar values of the target. A tree can be thought of as a collection of rules like `pickup_hour greater than 11` and `pickup_minute less than 31.0`. As you add more rules, those groups (called "leaf nodes") get smaller. In an extreme example, a model could create a tree with enough rules to place each record in the training data into its own group. That would probably take a lot of rules, and would be referred to as a "deep" tree.

Deep trees are problematic because their descriptions of the world are too specific to be useful on new data. Imagine training a classification model to predict whether or not visitors to a theme park will ride a particular rollercoaster. You could measure the time down to the millisecond that every guest's ticket is scanned at the entrance, and a model might learn a rule like *"if the guest has been to the park before and if the guest is older than 40 and younger than 41, and if the guest is staying at Hotel A and if the guest enters the park after 1:00:17.456 and if the guest enters the park earlier than 1:00:17.995, they will ride the rollercoaster"*. This is very very unlikely to ever match any future visitors, and if it does it's unlikely that this prediction will be very good unless you have some reason to believe that a visitor arriving at 1:00:18 instead of 1:00:17 really changes the probability that they'll ride that rollercoaster.

To prevent this situation (called "overfitting"), most tree-based machine learning algorithms accept parameters that control how deep the trees can get. `max_depth` is common, and says "don't create a rule more complex than this". In the example above, that rule has a depth of 7.

1. visiting the park
2. has been to the park before?
3. older than 40?
4. younger than 41?
5. staying at Hotel A?
6. entered the park after 1:00:17.456?
7. entered the park before 1:00:17.995?

Setting `max_depth = 5` would have prevented those weirdly-specific timing rules from ever being generated.

Choosing good values for this parameter is part art, part science, and is outside the scope of this tutorial.

</details>

In [ ]:
%%time

from cuml.dask.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=100, max_depth=10)
features = [c for c in taxi_train.columns if c != target_col]

rfc.fit(taxi_train[features], taxi_train[target_col])

<hr>

## Save Model

The distributed model objects from `cuml` cannot be pickled directly. Some of the model objects come with methods for generating a single-node equivalent (see ["Pickling cuML Models"](https://docs.rapids.ai/api/cuml/stable/pickling_cuml_models.html#Distributed-Model-Pickling)), but unfortunately `cuml.dask.ensemble.RandomForestClassifier` is not one of those.

Your best option for storing this model is to use a private method, `._concat_treelite_models()`. This produces a regular `cuml.ensemble.randomforestclassifier.RandomForestClassifier`, which you can use for scoring on local data.

In [ ]:
local_model = rfc._concat_treelite_models()

You can use this model to score on local data, like in a `pandas` data frame or `numpy` array.

In [ ]:
local_model.predict_proba(taxi_train.head(100)[features])

This model can also be saved for later use. There are several ways to do this, but `cloudpickle` is likely to give you the best experience. It handles some common drawbacks of the built-in `pickle` library.

`cloudpickle` can be used to write a Python object to bytes, and to create a Python object from that binary representation.

In [ ]:
import cloudpickle
import os

MODEL_PATH = "models"
if not os.path.exists(MODEL_PATH):
    os.makedirs(MODEL_PATH)

with open(f"{MODEL_PATH}/random_forest_dask_rapids.pkl", "wb") as f:
    cloudpickle.dump(local_model, f)

<hr>

## Calculate metrics on test set

Machine learning training tries to create a model which can produce useful results on new data that it didn't see during training. To test how well we've accomplished that in this example, read in another month of taxi data.

In [ ]:
taxi_test = dask_cudf.read_csv(
    "s3://nyc-tlc/trip data/yellow_tripdata_2019-02.csv",
    parse_dates=["tpep_pickup_datetime", "tpep_dropoff_datetime"],
    storage_options={"anon": True},
    assume_missing=True,
)

Before creating predictions on this new dataset, it has to be transformed in exactly the way that the original training data were prepared. Thankfully you've already wrapped that transformation logic in a function!

In [ ]:
taxi_test = prep_df(taxi_test, target_col=target_col)

`cuml` comes with many functions for calculating metrics that describe how well a model's predictions match the actual values. For a complete list, see [the cuml API docs](https://docs.rapids.ai/api/cuml/stable/api.html#metrics-regression-classification-and-distance) or run the code below.

In [ ]:
import cuml.metrics

[m for m in dir(cuml.metrics) if not m.startswith("_")]

This tutorial uses the `roc_auc_score` to evaluate the model. This metric measures the area under the [receiver operating characteristic](https://en.wikipedia.org/wiki/Receiver_operating_characteristic) curve. Values closer to 1.0 are desirable.

In [ ]:
from cuml.metrics import roc_auc_score

preds = rfc.predict_proba(taxi_test[features])[1]

As of this writing, `cuml.metrics.roc_auc_score` does not support Dask collections as inputs. The code below uses `.compute()` to create numpy arrays instead. This isn't ideal because it means that the amount of data you can score on is limited by the size of the instance you're running this code on. Hopefully support for Dask collections will be added in a future verison of `cuml`.

In [ ]:
roc_auc_score(taxi_test[target_col].compute(), preds.compute())

<hr>

## Next Steps

In this tutorial, you learned how to train a model for a binary classification task, using `cuml` and Dask. Training took around 3 seconds for a dataset that was 0.31 GB in memory, a huge improvement on the almost 9 minutes it took to [train the same model using `scikit-learn`](./rf-scikit.ipynb)!

You've reached the end of this set of exercises on using RAPIDS + Dask to accomplish distributed machine learning training using GPUs.

If you have data stored in Snowflake, consider exploring [the RAPIDS + Dask + Snowflake tutorial](../nyc-taxi-snowflake/rf-rapids-dask.ipynb).

If you want to try working with larger datasets, change the code in this notebook to use more data, or explore how to programmatically resize your Dask cluster by following the tips in ["Managing Dask Resources for Machine Learning Training"](https://www.saturncloud.io/docs/tips-and-tricks/training-on-dask).

<hr>